In [ ]:
# all - 4990

# 2555
# car - 1586
# human - 969

# 2435
# rink - 293
# dump_truck - 797
# bulldozer - 194
# excavator - 316
# mining_loader - 514
# truck_crane - 321

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from pipeline import make_pipeline_multy_label
from sklearn.metrics import recall_score

path_to_models = 'models'
if not os.path.isdir(path_to_models):
    os.mkdir(path_to_models)

# Step 1

In [2]:
X, Y = [], []
train_path = 'data/step_1'
W, H, D = 224,224,3
for img, label in make_pipeline_multy_label(train_path, 
                                            batch_size=1, 
                                            shuffle=True, 
                                            labels=True, 
                                            class_names = ['car_human',
                                                           'other'],
                                            im_size=(W, H, D)):
    X.append(np.array(img[0]))
    Y.append(np.array(label[0]))
    
s = 1 # 0.8

x_train, y_train = np.array(X[:int(len(Y) * s)]), np.array(Y[:int(len(Y) * s)])
x_test, y_test = np.array(X[int(len(Y) * s):]), np.array(Y[int(len(Y) * s):])
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((4990, 224, 224, 3), (4990, 2), (0,), (0,))

In [ ]:
xception = tf.keras.applications.Xception(
    include_top=0,
    weights='imagenet',
    input_tensor=None,
    input_shape=(W, H, D),
    pooling=None,
)

# flag = False
# for layer in xception.layers:
#     layer.trainable = flag
#     if 'block7' in layer.name:
#         flag = True
#         layer.trainable = True
#     else:
#         layer.trainable = False
        
xception.summary()

In [4]:
inp = Input(shape=(W,H,D))

layer = xception(inputs=inp)
layer = GlobalMaxPooling2D()(layer)
layer = Dense(units=128,
              activation='relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(units=2,
              activation='softmax')(layer)

model = Model(inputs=inp, outputs=layer)

model.compile(loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=[tf.keras.metrics.Recall()])

model.summary()
# tf.keras.utils.plot_model(model, show_shapes=1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                             

In [5]:
epochs = 5

history = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    validation_data=(x_test, y_test), 
                    batch_size=64)

Epoch 1/5
78/78 [==============================] - 37s 394ms/step - loss: 0.1403 - recall: 0.9415
Epoch 2/5
78/78 [==============================] - 29s 377ms/step - loss: 0.0145 - recall: 0.9950
Epoch 3/5
78/78 [==============================] - 30s 388ms/step - loss: 0.0046 - recall: 0.9982
Epoch 4/5
78/78 [==============================] - 29s 374ms/step - loss: 0.0021 - recall: 0.9996
Epoch 5/5
78/78 [==============================] - 28s 365ms/step - loss: 0.0031 - recall: 0.9990


In [7]:
model.save(f'{path_to_models}/xception_1.h5')

In [7]:
if len(x_test) > 0:
    data = []
    valid = np.argmax(model.predict(x_test), axis=1)

    val_accuracy = recall_score(np.argmax(y_test, axis=1), valid, average = "macro")
    print(f"Validation accuracy = {val_accuracy}")

Validation accuracy = 0.9859210563439889


# Step 2

In [9]:
X, Y = [], []
train_path = 'data/step_2'
for img, label in make_pipeline_multy_label(train_path, 
                                            batch_size=1, 
                                            shuffle=True, 
                                            labels=True, 
                                            class_names = ['car',
                                                           'human'],
                                            im_size=(W, H, D)):
    X.append(np.array(img[0]))
    Y.append(np.array(label[0]))
    
s = 1

x_train, y_train = np.array(X[:int(len(Y) * s)]), np.array(Y[:int(len(Y) * s)])
x_test, y_test = np.array(X[int(len(Y) * s):]), np.array(Y[int(len(Y) * s):])
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((2555, 224, 224, 3), (2555, 2), (0,), (0,))

In [ ]:
xception = tf.keras.applications.Xception(
    include_top=0,
    weights='imagenet',
    input_tensor=None,
    input_shape=(W, H, D),
    pooling=None,
)

# flag = False
# for layer in xception.layers:
#     layer.trainable = flag
# #     print(layer.name)
# #     print('block4_conv1' == layer.name)
#     if 'block7' in layer.name:
#         flag = True
#         layer.trainable = True
#     else:
#         layer.trainable = False
        
xception.summary()

In [11]:
inp = Input(shape=(W,H,D))

layer = xception(inputs=inp)
layer = GlobalMaxPooling2D()(layer)
layer = Dense(units=128,
              activation='relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(units=2,
              activation='softmax')(layer)

model = Model(inputs=inp, outputs=layer)

model.compile(loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=[tf.keras.metrics.Recall()])

model.summary()
# tf.keras.utils.plot_model(model, show_shapes=1)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_max_pooling2d_1 (Glo  (None, 2048)             0         
 balMaxPooling2D)                                                
                                                                 
 dense_2 (Dense)             (None, 128)               262272    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                           

In [12]:
epochs = 4

history = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    validation_data=(x_test, y_test), 
                    batch_size=64)

Epoch 1/4
40/40 [==============================] - 19s 399ms/step - loss: 0.1425 - recall_1: 0.9397
Epoch 2/4
40/40 [==============================] - 15s 379ms/step - loss: 0.0115 - recall_1: 0.9965
Epoch 3/4
40/40 [==============================] - 15s 367ms/step - loss: 0.0029 - recall_1: 0.9996
Epoch 4/4
40/40 [==============================] - 15s 370ms/step - loss: 0.0012 - recall_1: 0.9996


In [13]:
model.save(f'{path_to_models}/xception_2.h5')

In [12]:
if len(x_test) > 0:
    data = []
    valid = np.argmax(model.predict(x_test), axis=1)

    val_accuracy = recall_score(np.argmax(y_test, axis=1), valid, average = "macro")
    print(f"Validation accuracy = {val_accuracy}")

Validation accuracy = 1.0


# Step 3

In [15]:
X, Y = [], []
train_path = 'data/step_3'
for img, label in make_pipeline_multy_label(train_path, 
                                            batch_size=1, 
                                            shuffle=True, 
                                            labels=True, 
                                            class_names = ['truck_crane',
                                                           'excavator',
                                                           'dump_truck',
                                                           'mining_loader',
                                                           'rink',
                                                           'bulldozer'],
                                            im_size=(W, H, D)):
    X.append(np.array(img[0]))
    Y.append(np.array(label[0]))
    
s = 1

x_train, y_train = np.array(X[:int(len(Y) * s)]), np.array(Y[:int(len(Y) * s)])
x_test, y_test = np.array(X[int(len(Y) * s):]), np.array(Y[int(len(Y) * s):])
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((2435, 224, 224, 3), (2435, 6), (0,), (0,))

In [ ]:
xception = tf.keras.applications.Xception(
    include_top=0,
    weights='imagenet',
    input_tensor=None,
    input_shape=(W, H, D),
    pooling=None,
)

flag = False
for layer in xception.layers:
    layer.trainable = flag
    if 'block10' in layer.name:
        flag = True
        layer.trainable = True

xception.summary()

In [17]:
inp = Input(shape=(W,H,D))

layer = xception(inputs=inp)
layer = GlobalMaxPooling2D()(layer)
layer = Dense(units=128,
              activation='relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(units=6,
              activation='softmax')(layer)

model = Model(inputs=inp, outputs=layer)

model.compile(loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

model.summary()
# tf.keras.utils.plot_model(model, show_shapes=1)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_max_pooling2d_2 (Glo  (None, 2048)             0         
 balMaxPooling2D)                                                
                                                                 
 dense_4 (Dense)             (None, 128)               262272    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 6)                 774       
                                                           

In [18]:
epochs = 10

history = model.fit(x_train, 
                    y_train, 
#                     steps_per_epoch=50,
                    epochs=epochs, 
                    validation_data=(x_test, y_test), 
                    batch_size=64)

Epoch 1/10
39/39 [==============================] - 6s 95ms/step - loss: 1.1858 - acc: 0.5906 - precision: 0.7732 - recall_2: 0.4312
Epoch 2/10
39/39 [==============================] - 3s 87ms/step - loss: 0.2773 - acc: 0.9088 - precision: 0.9476 - recall_2: 0.8768
Epoch 3/10
39/39 [==============================] - 3s 87ms/step - loss: 0.1057 - acc: 0.9688 - precision: 0.9766 - recall_2: 0.9602
Epoch 4/10
39/39 [==============================] - 3s 87ms/step - loss: 0.0928 - acc: 0.9725 - precision: 0.9783 - recall_2: 0.9634
Epoch 5/10
39/39 [==============================] - 3s 87ms/step - loss: 0.0480 - acc: 0.9852 - precision: 0.9884 - recall_2: 0.9832
Epoch 6/10
39/39 [==============================] - 3s 87ms/step - loss: 0.0267 - acc: 0.9951 - precision: 0.9959 - recall_2: 0.9926
Epoch 7/10
39/39 [==============================] - 3s 87ms/step - loss: 0.0205 - acc: 0.9947 - precision: 0.9955 - recall_2: 0.9943
Epoch 8/10
39/39 [==============================] - 3s 87ms/step - lo

In [19]:
model.save(f'{path_to_models}/xception_3.h5')

In [22]:
if len(x_test) > 0:
    data = []
    valid = np.argmax(model.predict(x_test), axis=1)

    val_accuracy = recall_score(np.argmax(y_test, axis=1), valid, average = "macro")
    print(f"Validation accuracy = {val_accuracy}")

Validation accuracy = 0.9623998751404526
